<a href="https://colab.research.google.com/github/robson-rsp/datascience/blob/main/clustering/book_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Book recommender system
Este projeto é um sistema de recomendação de livros. A técnica utilizada é de filtragem colaborativa baseado em itens. Escolhi essa abordagem pois é mais eficiente que a baseada em usuários. O sistema vai recomendar um número x de livros parecidos com um livro referência. O único critério de escolha será as notas dadas pelos usuários.

Fonte: https://www.kaggle.com/datasets/rxsraghavagrawal/book-recommender-system

# Initial imports

In [ ]:
%pip install ipython-autotime --upgrade

In [2]:
from google.colab import drive, files
import pandas as pd
import warnings
drive.mount('/content/drive', force_remount=True)
warnings.filterwarnings("ignore")
%load_ext autotime

Mounted at /content/drive
time: 536 µs (started: 2023-04-16 19:49:48 +00:00)


In [30]:
books   = pd.read_csv("/content/drive/MyDrive/datasets/book-recommender-system/BX-Books.csv", sep=';', encoding='latin-1', on_bad_lines='skip')
ratings = pd.read_csv("/content/drive/MyDrive/datasets/book-recommender-system/BX-Book-Ratings.csv", sep=';', encoding='latin-1', on_bad_lines='skip')

time: 1.68 s (started: 2023-04-16 19:53:56 +00:00)


# EDA

In [ ]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


time: 14.9 ms (started: 2023-04-14 00:27:35 +00:00)


In [ ]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB
time: 309 ms (started: 2023-04-14 00:27:35 +00:00)


Com certeza não vou precisar de todos os atributos. Além disso, a técnica que usarei para criar o sistema dispensa transformações de tipos de dados.

In [ ]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


time: 7.31 ms (started: 2023-04-14 00:27:37 +00:00)


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
time: 394 ms (started: 2023-04-14 00:27:39 +00:00)


# Feature engineering

In [15]:
from scipy.sparse      import csr_matrix
from sklearn.base      import BaseEstimator, TransformerMixin
from sklearn.compose   import ColumnTransformer
from sklearn.pipeline  import Pipeline

import numpy as np

time: 593 µs (started: 2023-04-16 19:51:08 +00:00)


Vou remover os atributos que não serão necessário para a tarefa e renomear aqueles que ficarem.  Este projeto tem um processo de engenharia de atributos tão simples que não há necessidade de se criar classes transformadoras ou pipelines, por isso vou usar apenas funções. 

Vou criar dois conjuntos de dados, um para consulta, e outro para o treino. Este, conterá nas linhas os isbns dos livros, e nas colunas, os usuários. O conjunto será preenchido com as notas que cada usuário deu para cada livro.

In [31]:
books   = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

books   = books.rename(columns={'ISBN':'isbn', 'Book-Title':'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher'})
ratings = ratings.rename(columns={'User-ID':'user_id', 'ISBN':'isbn', 'Book-Rating':'rating'})

print(f'books:   {list(books.columns)}')
print(f'ratings: {list(ratings.columns)}')

books:   ['isbn', 'title', 'author', 'year', 'publisher']
ratings: ['user_id', 'isbn', 'rating']
time: 39.1 ms (started: 2023-04-16 19:54:03 +00:00)


**Etapa ##:** Identifico e seleciono os usuários que tenham feito mais de 50 avaliações e o livros que tenha recebido mais de 50. Os números foram escolhidos seguindo um critério pessoal.


In [32]:
def filter_rows(dataset, col_name, min_ratings):
  valid_rows = dataset[col_name].value_counts() > min_ratings
  valid_rows_ids = valid_rows[valid_rows].index
  mask = dataset[col_name].isin(valid_rows_ids)
  return dataset[mask]

ratings = filter_rows(ratings, 'user_id', 50)
ratings = filter_rows(ratings, 'isbn', 50)
print(ratings.shape)

(99223, 3)
time: 293 ms (started: 2023-04-16 19:54:07 +00:00)


**Etapa ##:** Crio uma nova tabela(isbn, usuário) que será preenchida com as notas que cada livro recebeu de cada usuário.

In [33]:
ratings_books = ratings.merge(books, on='isbn').drop(['title', 'author', 'year', 'publisher'], axis=1)
ratings_books = ratings_books.reset_index().drop('index', axis=1)
X = ratings_books.pivot(index='isbn', columns='user_id', values='rating')

time: 221 ms (started: 2023-04-16 19:54:10 +00:00)


**Etapa ##:** Encontro a média aritmética de cada linha e subtraio pela linha inteira. Isso vai resolver o problema de imputar nota 0 nos campos NaN. Assim, o modelo não vai pensar que nota 0 significa que a pessoa não gostou do filme uma vez que é média de cada linha.

In [34]:
row_means = np.nanmean(X, axis=1)
X = np.subtract(row_means.reshape(-1, 1), X)

time: 46.2 ms (started: 2023-04-16 19:54:13 +00:00)


**Etapa ##:** Substituo valores NaN por zero.

In [35]:
mask = np.isnan(X)
X[mask] = 0

time: 25.2 ms (started: 2023-04-16 19:54:15 +00:00)


**Etapa ##:** Transformo o dataframe em um numpy array.

In [36]:
X_numpy = X.values
print(f'X shape: {X_numpy.shape}')

X shape: (1059, 3150)
time: 1.2 ms (started: 2023-04-16 19:54:19 +00:00)


# Model training

Como o conjunto de dados não possui rótulo vou usar o modelo de agrupamento(clustering) NearestNeighbors. O seu critério de escolha para decidir quais vetores(linhas) estão mais próximos será a semelhança de cosenos.

In [37]:
from sklearn.metrics   import mean_absolute_error
from sklearn.neighbors import NearestNeighbors

time: 329 µs (started: 2023-04-16 19:54:19 +00:00)


Abaixo, a função mean_mae() fará a avaliação do modelo. Primeiro, para cada livro do dataset, o modelo encontrará um número x de outros livros que tenham um padrão de notas parecido com o seu. Essa semelhança é calculada pela classe NearestNeighbors. Para avaliar esses agrupamentos, para cada livro, vou comparar as notas que ele recebeu de cada usuário com as notas estipuladas pelo sistema. A distância entre a nota real com a calculada será armazenada em uma lista de erros. Por último, vou calcular a média e o desvio padrão desses erros.

In [38]:
def mean_mae(model, dataset):
  errors = list()
  for row in dataset:
    distances, indices = model.kneighbors(row.reshape(1, -1))
    distances = distances.flatten()
    indices   = indices.flatten()
    y_true, y_pred = predict_ratings(dataset.copy(), indices, distances)
    error = np.sqrt(mean_absolute_error(y_true, y_pred))
    errors.append(error)
  return errors

def predict_ratings(dataset, indices, distances):
  dataset[indices[1:]] = dataset[indices[1:]] * distances[1:].reshape(1, -1).T
  predictions  = dataset[indices[1:]].sum(axis=0) / distances[1:].sum()
  idx_nonzeros = dataset[indices[0]].nonzero()
  y_true = dataset[0][idx_nonzeros]
  y_pred = np.around(predictions[idx_nonzeros])
  return y_true, y_pred

time: 932 µs (started: 2023-04-16 19:54:21 +00:00)


## NearestNeighbors

O modelo abaixo vai determinar a nota de cada livro pela média ponderada dos 50 livros mais parecidos.

In [39]:
nn = NearestNeighbors(n_neighbors=50, metric='cosine')
nn.fit(X_numpy)

NearestNeighbors(metric='cosine', n_neighbors=50)

time: 7.73 ms (started: 2023-04-16 19:54:26 +00:00)


In [40]:
result = mean_mae(nn, X_numpy)
print(np.mean(result))

0.5401533060210272
time: 18.6 s (started: 2023-04-16 19:54:31 +00:00)


# Query

Obtenho uma amostra de dez livros que fazem parte daqueles que foram analisados pelo modelo.

In [63]:
isbns = ratings_books['isbn'].unique()
mask = books['isbn'].isin(isbns)
books[mask]['title'].sample(10)

9294                                       A Painted House
10569                               In Her Shoes : A Novel
20584    H Is for Homicide (Kinsey Millhone Mysteries (...
7997                                     Last Man Standing
3459      Harry Potter and the Chamber of Secrets (Book 2)
136                                  Before I Say Good-Bye
5962                                      Violets Are Blue
8143                                            Open House
4544       Heaven and Earth (Three Sisters Island Trilogy)
3022                              I Know This Much Is True
Name: title, dtype: object

time: 32 ms (started: 2023-04-16 20:04:47 +00:00)


Vou escolher o nome de um livro para que o modelo recomende 10 livros semelhantes em termos de nota.

In [65]:
def get_recommendation(model, dataset, neighbors, title):
  mask = dataset['title'].str.startswith(title)
  book_isbn = dataset[mask]['isbn']
  mask = neighbors.index.isin(book_isbn)
  book_reference = neighbors[mask].head(1)
  distances, indices = nn.kneighbors(book_reference.values)
  for i in indices[0][1:]:
    print(f"Title: {dataset.iloc[i]['title']}")

get_recommendation(nn, books, X, 'Violets Are Blue')

Title: Slow Waltz in Cedar Bend
Title: Coyote Waits (Joe Leaphorn/Jim Chee Novels)
Title: Life of Pi
Title: The Community in America
Title: The Firm
Title: The War in Heaven (Eternal Warriors)
Title: The Best Canadian Animal Stories: Classic Tales by Master Storytellers
Title: More Cunning Than Man: A Social History of Rats and Man
Title: If Love Were Oil, I'd Be About a Quart Low
Title: Chronique d'une mort annoncÃ?Â©e
Title: Twin Blessings (Love Inspired (Numbered))
Title: Die Mechanismen der Freude. ErzÃ?Â¤hlungen.
Title: The Gospel of Judas: A Novel
Title: Emma (Signet Classics (Paperback))
Title: PLEADING GUILTY
Title: The Bear and the Dragon
Title: Awakening
Title: The Hunted
Title: Tess of the D'Urbervilles (Wordsworth Classics)
Title: Night Sins
Title: The Robber Bride
Title: Deception Point
Title: Sturmzeit. Roman.
Title: Rule of the Bone : Novel, A
Title: Horus's Horrible Day (First Graders from Mars)
Title: Not a Day Goes By : A Novel
Title: Amy and Isabelle
Title: Postmorte